In [1]:
import numpy as np
from numpy import random as random
from scipy.special import erf
from math import sqrt,pi
from numpy import linalg as LA
import matplotlib.pyplot as plt

import email 
import os, sys, stat
import warnings

warnings.filterwarnings("ignore")
import shutil
from nltk.tokenize import word_tokenize, sent_tokenize
import re
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction import text
from sklearn.svm import LinearSVC


## Reading Email Data

In [2]:
Dir_train = 'CSDMC2010_SPAM/TRAINING'
stop = set(stopwords.words('english'))
sno = SnowballStemmer('english')

train_data = []
# k = 0
for name in sorted(os.listdir(Dir_train)):
#     if k>= 15000:

#         break
        
    file_data = ''
    file = Dir_train + '/' + name
    f = open(file,encoding="Latin-1",mode = 'r')
    data = f.read()
    
    ###### Reading and segmenting out email body and subject
    msg = email.message_from_string(data)
    payload = msg.get_payload()
    if type(payload) == type(list()) :
        payload = payload[0]
    sub = msg.get('subject')
    sub = str(sub)
    if type(payload) != type('') :
        payload = str(payload)
    
    ###### TOkenizing and preprocessing the the email contents    
    TAG_RE = re.compile(r'<[^>]+>')
    cleaned = TAG_RE.sub('', sub + ' ' + payload)
    cleaned = word_tokenize(cleaned)
    filt = [words for words in cleaned if words not in stop]
    filt = [words.lower() for words in filt if words.isalpha() and len(words)>2]
    filt = [sno.stem(words) for words in filt]
    
    ###### Preparing input for the count tokenizer
    for words in filt:
        file_data += ' ' + words
        
    train_data.append(file_data)
#     k += 1

## Forming label vector

In [3]:
label_file = 'CSDMC2010_SPAM/SPAMTrain.label'
label= np.genfromtxt(label_file,delimiter = ' ')[:,0]
y_label = np.zeros((len(label)))
for i in range(len(label)):
    if label[i] == 1.0:
        y_label[i] = 1
    else:
        y_label[i] = -1

## Getting Features

In [4]:
vectorizer = text.TfidfVectorizer()
x_train = vectorizer.fit_transform(train_data)
vocab = vectorizer.vocabulary_
x_train=x_train.toarray()

## BOW Features

In [5]:
vectorizer = text.CountVectorizer(binary=True,max_features=1000,lowercase=True)
x_train = vectorizer.fit_transform(train_data)
vocab = vectorizer.vocabulary_
x_train=x_train.toarray()

In [71]:
from sklearn.decomposition import PCA
import random as rd
print(x_train.shape[1])

print(x_train.shape)
print(y_label.shape)
y_label=y_label.reshape(-1,1)
X=np.concatenate((x_train,y_label),axis=1);
print(x_train.shape)

np.random.seed(2)
np.random.shuffle(X)

x_train=X[:,0:-1];
y_label=X[:,-1]

samples=1000;
xtrain=np.zeros((samples,x_train.shape[1]));
xtest=np.zeros((samples,x_train.shape[1]));


ind1=np.argwhere(y_label==1);
xtrain[0:int(samples/2),:]=x_train[ind1[0:int(samples/2),0],:]
xtest[0:int(samples/2),:]=x_train[ind1[int(samples/2):samples,0],:]

ind2=np.argwhere(y_label==-1);
xtrain[int(samples/2):samples,:]=x_train[ind2[0:int(samples/2),0],:]
xtest[int(samples/2):samples,:]=x_train[ind2[int(samples/2):samples,0],:]

ytrain=np.ones(samples)
ytrain[int(samples/2):samples]=-1

ytest=np.ones(samples)
ytest[int(samples/2):samples]=-1


# Perturbation vs Fscore
# clf = LinearSVC(max_iter=1000,dual=False, penalty='l1')
# clf.fit(xtrain,ytrain)


# d=[0,10,20,30,40,50,100]
# f1_svm=np.zeros(len(d));
# f1_nash=np.zeros(len(d));
# k=0;

# np.random.seed(2000)
# w1=np.random.multivariate_normal(mean, cov,1)
# w1=w1[0,:]



# for dmax in d:
# #Perturbing the test data
#     X=xtest[int(samples/2):samples,:]

#     for i in range(X.shape[0]):
#         rd.seed(i)
#         ind=rd.sample(range(1000),dmax)
#         for j in range(dmax):
#             if (X[i,ind[j]]==0):
#                 X[i,ind[j]]=1;
#             elif (X[i,ind[j]]==1):
#                 X[i,ind[j]]=0;
#     xtest[int(samples/2):samples,:]=X;
#     ypred1 = clf.predict(xtest);
#     f1_svm[k]=f1_score(ytest,ypred1);
#     ypred2=predict(w1,xtest)
#     f1_nash[k]=f1_score(ytest,ypred2);
#     k=k+1;

# plt.plot(d,f1_svm,label='SVM')
# plt.plot(d,f1_nash,label='RNash SVM')
# # plt.title('F1 Score vs d',fontsize=20)
# plt.xlabel('d',fontsize=16)
# plt.ylabel('F1 Score',fontsize=16)
# plt.legend()
# plt.show()


#-----------------------------------------------
# PCA 
#-----------------------------------------------

# pca = PCA(n_components=2)
# pca.fit(xtrain)
# xtrain=pca.transform(xtrain)
# xtest=pca.transform(xtest)

#Perturbing the test data
X=xtest[int(samples/2):samples,:]
dmax=100
for i in range(X.shape[0]):
    rd.seed(i)
    ind=rd.sample(range(1000),dmax)
    for j in range(dmax):
        if (X[i,ind[j]]==0):
            X[i,ind[j]]=1;
        elif (X[i,ind[j]]==1):
            X[i,ind[j]]=0;
xtest[int(samples/2):samples,:]=X;

# Plot of data
def get_plot(w,w1,w2,xtrain):
    xmin=xtrain[:,0].min()
    xmax=xtrain[:,0].max()
    xx=np.linspace(xmin,xmax,num=200)
    yy=(-(w[0]/w[1])*xx)#-(w[2]/w[1])
    yy1=(-(w1[0]/w1[1])*xx)#-(w[2]/w[1])
    yy2=(-(w2[0]/w2[1])*xx)#-(w[2]/w[1])
    
    
    
#     yy=(-(w[0]/w[1])*xx)-(w[2]/w[1])
#     yy1=(-(w1[0]/w1[1])*xx)-(w[2]/w[1])
#     yy2=(-(w2[0]/w2[1])*xx)-(w[2]/w[1])
    samples=xtrain.shape[0];
    
    plt.plot(xtrain[0:int(samples/2),0],xtrain[0:int(samples/2),1],'bo',label='Class 1')
    plt.plot(xtrain[int(samples/2):samples,0],xtrain[int(samples/2):samples,1],'yo',label='Class -1')
#     plt.plot(xmean[0,:],xmean[1,:],'ro',label='Mean')
    plt.plot(xx,yy,'k-')
    plt.plot(xx,yy1,'r-')
    plt.plot(xx,yy2,'r-')
    plt.xlabel('Feature-1')
    plt.ylabel('Feature-2')
    plt.legend(loc='best')
    plt.show()


# xtrain=np.c_[xtrain,np.ones((len(ytrain),1))];
# xtest=np.c_[xtest,np.ones((len(ytest),1))];

print(xtrain.shape[0])
print(xtest.shape)
print(ytest.shape)

1000
(4327, 1000)
(4327,)
(4327, 1000)
1000
(1000, 1000)
(1000,)


## Linear SVM for classification

In [20]:
def get_accuracy(ypred,ytest):
    acc=0;
    for i in range(len(ypred)):
        if(ypred[i]==ytest[i]):
            acc+=1;
    acc=acc/len(ypred);
    return acc*100

def get_class_accuracy(ypred,ytest):
    acc_1=0;
    acc_0=0;
    c1=0;
    c0=0;
    for i in range(len(ypred)):
        if(ytest[i]==1):
            c1+=1;
            if(ypred[i]==ytest[i]):
                acc_1+=1;
        if(ytest[i]==-1):
            c0+=1;
            if(ypred[i]==ytest[i]):
                acc_0+=1;       
    return acc_1,acc_0,c1,c0

def predict(W,xtest):
    def sign(x):
        if x>0:
            return 1
        else:
            return -1
    
    n,d=xtest.shape;
    y_out=np.zeros(n);
    for i in range(n):
        xi=xtest[i,:];
        y_out[i]=sign(W@xi);
    return y_out;

clf = LinearSVC(max_iter=1000,dual=False, penalty='l1')
clf.fit(xtrain,ytrain)
ypred = clf.predict(xtest)
cl1,cl0,ac1,ac0=get_class_accuracy(ypred,ytest)
print('Actual +1= ',ac1,' classified as +1= ',cl1)
print('Actual -1= ',ac0,' classified as -1= ',cl0)
print('Accuracy= ',acc)



Actual +1=  500  classified as +1=  472
Actual -1=  500  classified as -1=  354
Accuracy=  77.5


# Functions for Randomized Prediction Game Formulation

In [10]:
#---------------------------------------------------------------------------------------------
# Gradient Computation
#---------------------------------------------------------------------------------------------
def get_gradient(thetaL,thetaD,xtrain,ytrain,pl,pd):
    rl=1
    rd=pl/pd;
    n,m=xtrain.shape;
    
    mu_w=thetaL[:,0];
    mu_w_bar=thetaL[0:m-1,0];
    mu_b=thetaL[m-1,0];

    sigma_w=thetaL[:,1];
    sigma_w_bar=thetaL[0:m-1,1];
    sigma_b=thetaL[m-1,1];

    mu_x=thetaD[:,0:n];
    sigma_x=thetaD[:,n:2*n];


    # Gradient Computation of Learner's cost wrt learner's parameter(mean & variance)
    grad_l=np.zeros((m,2));
    mu_si=1-np.multiply(ytrain,((mu_w_bar.T@mu_x)+mu_b));
    sigma_si_2=np.square(sigma_w_bar).T@(np.square(mu_x)+np.square(sigma_x))+\
                np.square(mu_w_bar).T@np.square(sigma_x)+sigma_b**2;
        
    # Gradient of learner's cost function wrt Mean of W
    del_h_by_del_mu_si           = 0.5*(1-erf((-1/sqrt(2))*np.divide(mu_si,np.sqrt(sigma_si_2))));
    del_mu_si_by_del_mu_w        = -np.append(mu_x,np.ones((1,n)),axis=0)*ytrain;
    
    del_h_by_del_sigma_si2       = (1/2*sqrt(2*pi))*np.divide(np.exp((-0.5)*\
                                   np.divide(np.square(mu_si),sigma_si_2)),np.sqrt(sigma_si_2));
    
    del_sigma_si2_by_del_mu_w    = np.append((2*np.square(sigma_x))*mu_w_bar[:,np.newaxis],np.zeros((1,n)),axis=0)      

    del_Ll_by_del_mu_w           = pl*np.append(mu_w_bar,0)+np.sum((del_mu_si_by_del_mu_w*del_h_by_del_mu_si),axis=1)+\
                                   np.sum((del_sigma_si2_by_del_mu_w*del_h_by_del_sigma_si2),axis=1)

    # Gradient of learner's cost function wrt variance of W
    del_sigma_si2_by_del_sigma_w =2*((np.append(np.square(sigma_x)+np.square(mu_x),\
                                            np.ones((1,n)),axis=0))*sigma_w[:,np.newaxis]);


    del_Ll_by_del_sigma_w        = pl*np.append(sigma_w_bar,0)+np.sum((del_sigma_si2_by_del_sigma_w*\
                                   del_h_by_del_sigma_si2),axis=1)   


    grad_l[:,0]=del_Ll_by_del_mu_w ;
    grad_l[:,1]=del_Ll_by_del_sigma_w;
    # Gradient Computation of Data Generator's cost wrt its parameters
    grad_dg=np.zeros((m-1,2*n));
    
    for i in range(n):
        xi_hat=xtrain[i,0:m-1];
        mu_xi=mu_x[:,i];
        sigma_xi=sigma_x[:,i];

        mu_ti                       = 1+np.multiply(ytrain[i],((mu_w_bar.T@mu_xi)+mu_b));
        sigma_ti_2                  = np.square(sigma_w_bar).T@(np.square(mu_xi)+np.square(sigma_xi))+\
                                      np.square(mu_w_bar).T@np.square(sigma_xi)+sigma_b**2;

        del_h_by_del_mu_ti          = 0.5*(1-erf((-1/sqrt(2))*np.divide(mu_ti,np.sqrt(sigma_ti_2))));
        del_h_by_del_sigma_ti2      = (1/2*sqrt(2*pi))*np.divide(np.exp((-0.5)*\
                                       np.divide(np.square(mu_ti),sigma_ti_2)),np.sqrt(sigma_ti_2));

        del_mu_ti_by_del_mu_xi      = ytrain[i]*mu_w_bar;
        del_sigma_ti2_del_mu_xi     = np.multiply(2*np.square(sigma_w_bar),mu_xi)

        del_sigma_ti2_del_sigma_xi  = np.multiply(2*sigma_xi,(np.square(sigma_w_bar)+np.square(mu_w_bar)))

        del_Ld_by_del_mu_xi         = (mu_xi-xi_hat)+pd*((del_h_by_del_mu_ti*del_mu_ti_by_del_mu_xi)+\
                                      (del_h_by_del_sigma_ti2*del_sigma_ti2_del_mu_xi))

        del_Ld_by_del_sigma_xi      = sigma_xi+pd*(del_h_by_del_sigma_ti2*del_sigma_ti2_del_sigma_xi)
        
        grad_dg[:,i]   = del_Ld_by_del_mu_xi;
        grad_dg[:,n+i] = del_Ld_by_del_sigma_xi;
        grad_dg=rd*grad_dg;
        # grad_l is the gradientof learner's cost wrt thetaL. It will have same no. of element as thetaD 
        # grad_dg is the gradient of data generator's cost wrt thetaD.
    return grad_l,grad_dg;

#---------------------------------------------------------------------------------------------
# Bound
#---------------------------------------------------------------------------------------------

def bound(temp_l,temp_dg): 
    # Initializing the upper and lower bound for all the parameters
    # Learner's mean
    W=0.01;
    mu_l_max=W;
    mu_l_min=-W;
    # Data Generator's mean
    mu_dg_max=1;
    mu_dg_min=0;
    # Learner's variance
    var_l_max=10**(-3);
    sigma_l_max=sqrt(var_l_max);
    var_l_min=10**(-6);
    sigma_l_min=sqrt(var_l_min);
    # Data Generator's Variance
    var_dg_max=0.5;
    sigma_dg_max=sqrt(var_dg_max)
    var_dg_min=10**(-3);
    sigma_dg_min=sqrt(var_dg_min)

    # Bounding mean of learner 
    ind=np.where(temp_l[:,0]>mu_l_max)
    temp_l[ind,0]=mu_l_max;
    ind=np.argwhere(temp_l[:,0]<mu_l_min)
    temp_l[ind,0]=mu_l_min;

    # Bounding std dev of learner
    ind=np.where(temp_l[:,1]>sigma_l_max)
    temp_l[ind,1]=sigma_l_max;
    ind=np.where(temp_l[:,1]<sigma_l_min)
    temp_l[ind,1]=sigma_l_min;

    x=temp_dg.shape
    n=int(x[1]/2);

    # # Bounding mean of data generator
    for i in range(n):
        ind=np.where(temp_dg[:,i]>mu_dg_max)
        temp_dg[ind,i]=mu_dg_max;
        ind=np.where(temp_dg[:,i]<mu_dg_min)
        temp_dg[ind,i]=mu_dg_min
    # # Bounding std dev of data generator
    for i in range(n,2*n):
        ind=np.where(temp_dg[:,i]>sigma_dg_max)
        temp_dg[ind,i]=sigma_dg_max;
        ind=np.where(temp_dg[:,i]<sigma_dg_min)
        temp_dg[ind,i]=sigma_dg_min
    return temp_l,temp_dg;

def get_norm(d_l,d_dg):
    norm1=np.square(d_l).sum()+np.square(d_dg).sum()
    return norm1

def get_dot(grad_l,grad_dg,d_l,d_dg):
    temp=np.multiply(grad_l,d_l).sum()+np.multiply(grad_dg,d_dg).sum();
    return temp

#---------------------------------------------------------------------------------------------
# Extra Gradient
#---------------------------------------------------------------------------------------------

def extraGradient(xtrain,ytrain,sigma,beta,pl,pd,eps):
    n,m=xtrain.shape;
    # Initializing stategy sets of both the players. For learner strategy is parameters of distribution of W
    # Parameters include mean and standard deviation of multi dimension gaussian from which W is sampled.
    # Parameters are stored in two columns. First column corresponds to mean vector(mu_w) of m dimension and 
    # second column is std dev vector(sigma_w)
    thetaL=random.rand(m,2);# concatenation of mu_w and sigma_w
    # Strategy set of data generator consist of distribution parameters for each sample in the training set. For each sample
    # We have one mean vector and one std dev vector. For n samples we will have 2n vectors arranged in columns of thetaD
    # First n columns corresponds to mean vector of n samples and last n columns corresponds to std dev vector of n samples
    thetaD=random.rand(m-1,2*n);# n columns for mu and n columns for sigma

    thetaL,thetaD=bound(thetaL,thetaD)
    
    
    k=0;

    flag=1;
    #Extra Gradient Algorithm
    while(flag==1):
        # Step 5
#         get_plot(thetaL[:,0],thetaL[:,0]-thetaL[:,1],thetaL[:,0]+thetaL[:,1],xtrain)
#         print(thetaL)
        grad_l,grad_dg=get_gradient(thetaL,thetaD,xtrain,ytrain,pl,pd);

        temp_l=thetaL-grad_l;
        temp_dg=thetaD-grad_dg;

        l_bounded,dg_bounded=bound(temp_l,temp_dg)

        d_l=l_bounded-thetaL;
        d_dg=dg_bounded-thetaD;
        
#         ## Print norm of d
        d_norm=get_norm(d_l,d_dg)
#         print('Norm of d= ',d_norm)
        # Step 6
        flag1=1
        p=1
        while (flag1==1):
            print('p=',p)
            t=beta**p;
            p+=1;
            
            thetaL_bar=thetaL+t*d_l;
            thetaD_bar=thetaD+t*d_dg;

            grad_l_bar,grad_dg_bar=get_gradient(thetaL_bar,thetaD_bar,xtrain,ytrain,pl,pd);
            d_norm=get_norm(d_l,d_dg);
            g_dot_d=get_dot(grad_l_bar,grad_dg_bar,d_l,d_dg);
            if ((-g_dot_d) >= (sigma*d_norm)):
                flag1=0

#         print('Norm of theta_bar-theta= ',get_norm(thetaL_bar-thetaL,thetaD_bar-thetaD))
        # Step 7
#         thetaL_bar=thetaL+t*d_l;
#         thetaD_bar=thetaD+t*d_dg;
#         grad_l_bar,grad_dg_bar=get_gradient(thetaL_bar,thetaD_bar,xtrain,ytrain,pl,pd);

        eta=(-t/get_norm(grad_l_bar,grad_dg_bar))*get_dot(grad_l_bar,grad_dg_bar,d_l,d_dg)
#         print(grad_l_bar)
        # Step 8
        temp_l=thetaL-eta*grad_l_bar;
        temp_dg=thetaD-eta*grad_dg_bar;

        new_thetaL,new_thetaD=bound(temp_l,temp_dg)

        # Step 9
        k=k+1
        print('iteration= ',k)

        # Step 10
        error=get_norm((new_thetaL-thetaL),(new_thetaD-thetaD))
        print('error=',error)
        thetaL=np.copy(new_thetaL);
        thetaD=np.copy(new_thetaD);

        if (error<eps):
            flag=0;
        if (k>50):
            flag=0;
    return thetaL    

## Parameter Tuning and Testing

In [15]:
# Rows of xtrain corresponds to samples and columns corresponds to features
sigma=0.01
beta=0.5
pl=1
pd=1
eps=0.001;# Stopping threshold
w=extraGradient(xtrain,ytrain,sigma,beta,pl,pd,eps)


p= 1
iteration=  1
error= 23883.4018318
p= 1
iteration=  2
error= 23.1685411556
p= 1
iteration=  3
error= 5335.57670596
p= 1
iteration=  4
error= 3038.68335894
p= 1
iteration=  5
error= 1550.77954006
p= 1
iteration=  6
error= 908.938810372
p= 1
iteration=  7
error= 470.32309666
p= 1
iteration=  8
error= 332.178540467
p= 1
iteration=  9
error= 272.729352596
p= 1
iteration=  10
error= 176.839285638
p= 1
iteration=  11
error= 153.998925944
p= 1
iteration=  12
error= 108.445737901
p= 1
iteration=  13
error= 102.377051548
p= 1
iteration=  14
error= 83.612093814
p= 1
iteration=  15
error= 70.3123584492
p= 1
iteration=  16
error= 57.1877879971
p= 1
iteration=  17
error= 49.3063904236
p= 1
iteration=  18
error= 41.3943228211
p= 1
iteration=  19
error= 37.7855983482
p= 1
iteration=  20
error= 32.6381535965
p= 1
iteration=  21
error= 28.6690877867
p= 1
iteration=  22
error= 25.0716526418
p= 1
iteration=  23
error= 22.2924033461
p= 1
iteration=  24
error= 19.683778754
p= 1
iteration=  25
error= 1

In [72]:

# Randomly selecting one sample from normal distribution
mean=w[:,0]
cov=np.diagflat(w[:,1])
np.random.seed(2000)
# np.random.seed(2000)# perturbed data
w1=np.random.multivariate_normal(mean, cov,1)
w1=w1[0,:]

# ypred=predict(w1[:,0],xtest)
ypred=predict(w1,xtest)
# print(ypred)
acc=get_accuracy(ypred,ytest)
cl1,cl0,ac1,ac0=get_class_accuracy(ypred,ytest)
print('Actual +1= ',ac1,' classified as +1= ',cl1)
print('Actual -1= ',ac0,' classified as -1= ',cl0)
print('Accuracy= ',acc)



Actual +1=  500  classified as +1=  371
Actual -1=  500  classified as -1=  404
Accuracy=  77.5
